In [1]:
# Gerekli kütüphanelerin eklenmesi
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import string
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
import lightgbm as lgb
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import resample
from imblearn.over_sampling import SMOTE
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem import WordNetLemmatizer
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, LSTM, GRU, Dense

In [2]:
# Veri setinin yüklenmesi
data = pd.read_csv('mtsamples.csv')

In [3]:
# Fonksiyon: Cümle ve kelime sayısını hesapla
def get_sentence_word_count(text_list):
    sent_count = 0
    word_count = 0
    vocab = {}
    
    for text in text_list:
        sentences = sent_tokenize(str(text).lower())
        sent_count += len(sentences)
        
        for sentence in sentences:
            words = word_tokenize(sentence)
            
            for word in words:
                if word in vocab:
                    vocab[word] += 1
                else:
                    vocab[word] = 1
    
    word_count = len(vocab.keys())
    return sent_count, word_count

In [4]:
# Veri kümesini yükle
clinical_text_df = pd.read_csv("mtsamples.csv")

# NaN transkripsiyonları temizle
clinical_text_df = clinical_text_df[clinical_text_df['transcription'].notna()]

# Kategori sayısı 50'nin altında olan transkripsiyonları temizle
filtered_data_categories = clinical_text_df.groupby('medical_specialty').filter(lambda x: x.shape[0] > 50)

In [5]:
# Veri ön işleme ve temizleme
def clean_text(text):
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = ''.join([w for w in text if not w.isdigit()])
    REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
    text = text.lower()
    text = REPLACE_BY_SPACE_RE.sub('', text)
    return text

def lemmatize_text(text):
    wordlist = []
    lemmatizer = WordNetLemmatizer()
    sentences = sent_tokenize(text)

    intial_sentences = sentences[:1]
    final_sentences = sentences[-2:-1]

    for sentence in intial_sentences:
        words = word_tokenize(sentence)
        wordlist.extend(lemmatizer.lemmatize(word) for word in words)

    for sentence in final_sentences:
        words = word_tokenize(sentence)
        wordlist.extend(lemmatizer.lemmatize(word) for word in words)

    return ' '.join(wordlist)

filtered_data_categories['transcription'] = filtered_data_categories['transcription'].apply(lemmatize_text)
filtered_data_categories['transcription'] = filtered_data_categories['transcription'].apply(clean_text)

In [6]:
# Veri kümesini eğitim ve test setlerine ayırma
X = filtered_data_categories['transcription']
y = filtered_data_categories['medical_specialty']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [7]:
# TF-IDF vektörleştirme
vectorizer_tfidf = TfidfVectorizer(sublinear_tf=True, encoding='utf-8', decode_error='ignore')
X_train_tfidf = vectorizer_tfidf.fit_transform(X_train)
X_test_tfidf = vectorizer_tfidf.transform(X_test)

In [8]:
# SMOTE ile dengesiz sınıfları dengeleme
smote = SMOTE(random_state=42)
X_train_smote_tfidf, y_train_smote_tfidf = smote.fit_resample(X_train_tfidf, y_train)

In [9]:
# Naive Bayes
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train_smote_tfidf, y_train_smote_tfidf)
y_pred_nb_tfidf = nb_classifier.predict(X_test_tfidf)

In [10]:
# Random Forest
rf_classifier = RandomForestClassifier(random_state=1)
rf_classifier.fit(X_train_smote_tfidf, y_train_smote_tfidf)
y_pred_rf_tfidf = rf_classifier.predict(X_test_tfidf)

In [11]:
# XGBoost
xgb_classifier = XGBClassifier(random_state=1)
# Encode labels using LabelEncoder
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train_smote_tfidf)
xgb_classifier.fit(X_train_smote_tfidf, y_train_encoded)
# Ensure that classes match
print("Expected Classes:", xgb_classifier.classes_)
y_pred_xgb_tfidf = xgb_classifier.predict(X_test_tfidf)

Expected Classes: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20]


In [12]:
# LightGBM
lgb_classifier = lgb.LGBMClassifier(random_state=1)
lgb_classifier.fit(X_train_smote_tfidf, y_train_smote_tfidf)
y_pred_lgb_tfidf = lgb_classifier.predict(X_test_tfidf)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.153001 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 194540
[LightGBM] [Info] Number of data points in the train set: 18270, number of used features: 4200
[LightGBM] [Info] Start training from score -3.044522
[LightGBM] [Info] Start training from score -3.044522
[LightGBM] [Info] Start training from score -3.044522
[LightGBM] [Info] Start training from score -3.044522
[LightGBM] [Info] Start training from score -3.044522
[LightGBM] [Info] Start training from score -3.044522
[LightGBM] [Info] Start training from score -3.044522
[LightGBM] [Info] Start training from score -3.044522
[LightGBM] [Info] Start training from score -3.044522
[LightGBM] [Info] Start training from score -3.044522
[LightGBM] [Info] Start training from score -3.044522
[LightGBM] [Info] Start training from score -3.04

In [13]:
# Çıktıları CSV dosyalarına kaydetme
pd.DataFrame(y_pred_nb_tfidf).to_csv('y_pred_nb_tfidf.csv', index=False)
pd.DataFrame(y_pred_rf_tfidf).to_csv('y_pred_rf_tfidf.csv', index=False)
pd.DataFrame(y_pred_xgb_tfidf).to_csv('y_pred_xgb_tfidf.csv', index=False)
pd.DataFrame(y_pred_lgb_tfidf).to_csv('y_pred_lgb_tfidf.csv', index=False)

In [14]:
# NER Kullanımı
# Gerekli kütüphanelerin eklenmesi
import spacy
import en_ner_bc5cdr_md as ner_model
import pandas as pd

In [15]:
# MTSamples veri setini yükle
mtsample_df = pd.read_csv('mtsamples.csv')

# Boş olmayan transkripsiyonları seç
mtsample_df.dropna(subset=['transcription'], inplace=True)

# NER modelini yükle
nlp = ner_model.load()

# Örnek transkripsiyonları al
sample_transcriptions = mtsample_df['transcription'].sample(n=5, random_state=42)

# NER işlemi
for transcription in sample_transcriptions:
    doc = nlp(transcription)
    
    print("\nTranscription Text:")
    print(transcription)
    
    print("\nNamed Entities:")
    print("TEXT\t\tSTART\tEND\tENTITY TYPE")
    for ent in doc.ents:
        print(f"{ent.text}\t\t{ent.start_char}\t{ent.end_char}\t{ent.label_}")

c:\Users\cihat\Downloads\Extracting Medical Information From Clinical Text With NLP\venv\Lib\site-packages\spacy\language.py:2141: FutureWarning: Possible set union at position 6328
  deserializers["tokenizer"] = lambda p: self.tokenizer.from_disk(  # type: ignore[union-attr]



Transcription Text:
HISTORY OF PRESENT ILLNESS:,  The patient is well known to me for a history of iron-deficiency anemia due to chronic blood loss from colitis.  We corrected her hematocrit last year with intravenous (IV) iron.  Ultimately, she had a total proctocolectomy done on 03/14/2007 to treat her colitis.  Her course has been very complicated since then with needing multiple surgeries for removal of hematoma.  This is partly because she was on anticoagulation for a right arm deep venous thrombosis (DVT) she had early this year, complicated by septic phlebitis.,Chart was reviewed, and I will not reiterate her complex history.,I am asked to see the patient again because of concerns for coagulopathy.,She had surgery again last month to evacuate a pelvic hematoma, and was found to have vancomycin resistant enterococcus, for which she is on multiple antibiotics and followed by infectious disease now.,She is on total parenteral nutrition (TPN) as well.,LABORATORY DATA:,  Labs today 

In [29]:
# Derin Öğrenme Modeli
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(y_train)

model = Sequential()
model.add(Embedding(input_dim=len(vectorizer_tfidf.get_feature_names_out()), output_dim=32, input_length=X_train_tfidf.shape[1]))
model.add(Conv1D(128, 5, activation='relu'))
model.add(LSTM(64, return_sequences=False))  # Changed here
model.add(Dense(len(label_encoder.classes_), activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train_tfidf, encoded_labels, epochs=10, batch_size=32, validation_split=0.2)
y_pred_dnn_tfidf = model.predict_classes(X_test_tfidf)

Epoch 1/10
92/92 [==============================] - 630s 7s/step - loss: 2.7377 - accuracy: 0.2299 - val_loss: 2.6646 - val_accuracy: 0.2554
Epoch 2/10
92/92 [==============================] - 633s 7s/step - loss: 2.6874 - accuracy: 0.2319 - val_loss: 2.6633 - val_accuracy: 0.2554
Epoch 3/10
92/92 [==============================] - 630s 7s/step - loss: 2.6805 - accuracy: 0.2319 - val_loss: 2.6688 - val_accuracy: 0.2554
Epoch 4/10
92/92 [==============================] - 623s 7s/step - loss: 2.6812 - accuracy: 0.2319 - val_loss: 2.6683 - val_accuracy: 0.2554
Epoch 5/10
92/92 [==============================] - 619s 7s/step - loss: 2.6792 - accuracy: 0.2319 - val_loss: 2.6583 - val_accuracy: 0.2554
Epoch 6/10
92/92 [==============================] - 624s 7s/step - loss: 2.6792 - accuracy: 0.2319 - val_loss: 2.6593 - val_accuracy: 0.2554
Epoch 7/10
92/92 [==============================] - 626s 7s/step - loss: 2.6805 - accuracy: 0.2319 - val_loss: 2.6581 - val_accuracy: 0.2554
Epoch 8/10
92

AttributeError: 'Sequential' object has no attribute 'predict_classes'

In [31]:
model.save('my_model.h5')

c:\Users\cihat\Downloads\Extracting Medical Information From Clinical Text With NLP\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [32]:
# Performans Metrikleri
def evaluate_model(y_true, y_pred, classes):
    cm = confusion_matrix(y_true, y_pred, labels=classes)
    accuracy = accuracy_score(y_true, y_pred)
    classification_rep = classification_report(y_true, y_pred, labels=classes)
    
    print("Confusion Matrix:")
    print(cm)
    print("\nAccuracy:", accuracy)
    print("\nClassification Report:")
    print(classification_rep)

In [33]:
print(evaluate_model)

<function evaluate_model at 0x000001D35129BA60>
